In [31]:
import numpy as np
import pandas as pd
import ast
from sentence_transformers import util
import matplotlib.pyplot as plt
from itertools import chain


In [32]:
expunations_base = pd.read_csv('jokes_base.csv', encoding='utf-8')


In [33]:
expunations_base = expunations_base.drop(['sentence_1', 'word_1', 'sentence_2', 'word_2'], axis=1)


In [34]:
expunations_pairs = pd.read_csv('expunations_chatgpt_generated_sentences.csv', encoding='utf-8', sep=';')


In [35]:
expunations_base

,ID,explanation,keywords,joke,funniness_average,filtered_funniness_average
0,hom_657,['the phrase dish it out means to criticizie a...,"[[""waitress's"", 'dish it out'], [''], [''], ['...",She was only a Waitress ' s daughter but she s...,0.6,1.000000
1,het_1635,"['', 'This is a play on words. The words ""pane...","[[''], ['glazier', 'window', 'feel your pane',...",When the glazier was sent to the hospital room...,1.2,2.000000
2,hom_1510,"['sandpaper is normally rough on one side ', '...","[['inventor', 'sandpaper', 'rough'], ['invento...",The inventor of sandpaper had a rough time of ...,1.6,1.600000
3,hom_955,"['', '', '', 'A contortionist is someone who c...","[[''], [''], [''], ['contortionists', 'die', '...","OLD CONTORTIONISTS never die , they just meet ...",0.2,1.000000
4,hom_1505,"['', 'This is a play on words. The word ""party...","[[''], ['whose place to have a party at', 'par...",When you can ' t decide with your friends whos...,0.4,0.666667
...,...,...,...,...,...,...
1890,hom_2076,"['', ""This is a pun on the phrase 'stroke of g...","[[''], ['Da Vinci', 'stroke', 'genius'], ['Leo...",Leonardo Da Vinci could use a paint brush to m...,0.4,0.500000
1891,hom_1437,['driven can mean to drive a car or it can me ...,"[['cars', 'driven'], ['cars', 'driven', 'eden'...",The earliest mention of cars is in the Bible w...,1.0,1.250000
1892,het_1530,"['son sounds like sun ', ""This is a pun on 'su...","[['doting', 'son'], ['parents', 'son', 'worshi...",Some doting parents are son worshipers .,0.8,1.000000
1893,het_100,"['lion sounds like line ', ""This is a pun on t...","[['zoo', 'lion'], ['phone', 'zoo', 'lion'], ['...",I phoned the zoo but the lion was busy .,1.6,1.600000


In [36]:
expunations_pairs

,ID,expression1,expression2,sentence1,sentence2
0,hom_657,dish it out,dish it out,The waitress had to dish it out to all the cus...,"During the feedback session, she was able to d..."
1,het_1635,pane,pain,The glazier carefully installed the new pane i...,The patient experienced severe pain after the ...
2,hom_1510,rough,rough,He had a rough day dealing with the challenges.,The sandpaper is rough and scratches the surface.
3,hom_955,meet their end,meet their end,Everyone will meet their end eventually.,The contortionist could effortlessly meet thei...
4,hom_1505,party,party,We are hosting a party at our home this Saturday.,The Liberal party announced its new policy age...
...,...,...,...,...,...
1886,hom_1848,out of the picture,out of the picture,"After the company's restructuring, the marketi...","During the family reunion, Uncle Joe was out o..."
1887,hom_888,do a number on it,do a number on it,The math professor did a number on the blackbo...,"The vandals did a number on the storefront, ca..."
1888,het_1504,paragon,tarragon,She is a paragon of virtue in her community.,The chef added fresh tarragon to the sauce for...
1889,het_459,patiently,patient,Tom waited patiently to see the doctor.,"Tom, as a patient, was eager to see the doctor."


In [37]:
expunations = pd.merge(expunations_base, expunations_pairs, on='ID', how='inner')

In [38]:
expunations

,ID,explanation,keywords,joke,funniness_average,filtered_funniness_average,expression1,expression2,sentence1,sentence2
0,hom_657,['the phrase dish it out means to criticizie a...,"[[""waitress's"", 'dish it out'], [''], [''], ['...",She was only a Waitress ' s daughter but she s...,0.6,1.000000,dish it out,dish it out,The waitress had to dish it out to all the cus...,"During the feedback session, she was able to d..."
1,het_1635,"['', 'This is a play on words. The words ""pane...","[[''], ['glazier', 'window', 'feel your pane',...",When the glazier was sent to the hospital room...,1.2,2.000000,pane,pain,The glazier carefully installed the new pane i...,The patient experienced severe pain after the ...
2,hom_1510,"['sandpaper is normally rough on one side ', '...","[['inventor', 'sandpaper', 'rough'], ['invento...",The inventor of sandpaper had a rough time of ...,1.6,1.600000,rough,rough,He had a rough day dealing with the challenges.,The sandpaper is rough and scratches the surface.
3,hom_955,"['', '', '', 'A contortionist is someone who c...","[[''], [''], [''], ['contortionists', 'die', '...","OLD CONTORTIONISTS never die , they just meet ...",0.2,1.000000,meet their end,meet their end,Everyone will meet their end eventually.,The contortionist could effortlessly meet thei...
4,hom_1505,"['', 'This is a play on words. The word ""party...","[[''], ['whose place to have a party at', 'par...",When you can ' t decide with your friends whos...,0.4,0.666667,party,party,We are hosting a party at our home this Saturday.,The Liberal party announced its new policy age...
...,...,...,...,...,...,...,...,...,...,...
1885,hom_2076,"['', ""This is a pun on the phrase 'stroke of g...","[[''], ['Da Vinci', 'stroke', 'genius'], ['Leo...",Leonardo Da Vinci could use a paint brush to m...,0.4,0.500000,stroke of genius,stroke of genius,"Her presentation was truly a stroke of genius,...","Using a fine brush, the artist made a delicate..."
1886,hom_1437,['driven can mean to drive a car or it can me ...,"[['cars', 'driven'], ['cars', 'driven', 'eden'...",The earliest mention of cars is in the Bible w...,1.0,1.250000,driven out,driven out,The manager was driven out of the company afte...,He was driven out of town in his new sports car.
1887,het_1530,"['son sounds like sun ', ""This is a pun on 'su...","[['doting', 'son'], ['parents', 'son', 'worshi...",Some doting parents are son worshipers .,0.8,1.000000,son,sun,She is proud of her son for winning the compet...,The sun shines brightly during the summer months.
1888,het_100,"['lion sounds like line ', ""This is a pun on t...","[['zoo', 'lion'], ['phone', 'zoo', 'lion'], ['...",I phoned the zoo but the lion was busy .,1.6,1.600000,lion,line,The lion roamed freely in the zoo.,The phone line was busy during the call.


In [39]:
expunations.to_csv("expunations_complete_with_score_and_generated_pairs.csv", encoding='utf-8', index=False)